# Weighted Average Basic Recommendations

In [1]:
import numpy as np
import pandas as pd

In [38]:
def change_name(x):
    y = x.split(", The")
    if(len(y)==1):
        return x
    else:
        y = "".join([word for word in y])
        y = "The " + y
        return y

In [82]:
ratings = pd.read_csv("../../Datasets/ratings.csv")
movies = pd.read_csv("../../Datasets/movies.csv")
movies['title'] = movies['title'].apply(lambda x: change_name(x))

In [11]:
ratings = ratings.merge(movies)
ratings.dropna(thresh=10, inplace=True, axis=1)
ratings.fillna(0,axis=1)

grouped_ratings = ratings.groupby(by='movieId')['rating'].agg(['mean','count']).rename(columns={'mean':'average_rating','count':'vote_count'})
grouped_ratings = grouped_ratings[grouped_ratings['vote_count']>100]
grouped_ratings['standardized_vote_count'] = (5/grouped_ratings['vote_count'].max())*grouped_ratings['vote_count']
grouped_ratings[['average_rating','standardized_vote_count']].merge(movies,on='movieId').to_csv("../Datasets/average_ratings.csv")

In [184]:
grouped_ratings['weighted average'] = 0.5*grouped_ratings['average rating'] + 0.5*grouped_ratings['standardized vote count']
grouped_ratings = grouped_ratings['weighted average'].sort_values(ascending=False)
# final_movies = movies.set_index('movieId').loc[grouped_ratings['weighted average'].sort_values(ascending=False).index.values,:].merge(grouped_ratings['weighted average'], on='movieId')
# final_movies.head(20)
movies.set_index('movieId').loc[grouped_ratings.index.values,:].merge(grouped_ratings, on='movieId')

,title,genres,weighted average
movieId,,,
318,"Shawshank Redemption, The (1994)",Crime Drama,4.623326
356,Forrest Gump (1994),Comedy Drama Romance War,4.582067
296,Pulp Fiction (1994),Comedy Crime Drama Thriller,4.431361
2571,"Matrix, The (1999)",Action Sci-Fi Thriller,4.208685
593,"Silence of the Lambs, The (1991)",Crime Horror Thriller,4.200706
...,...,...,...
39,Clueless (1995),Comedy Romance,2.436908
586,Home Alone (1990),Children Comedy,2.379304
185,"Net, The (1995)",Action Crime Thriller,2.371153


In [163]:
final_movies.head()
final_movies['genres list'] = final_movies['genres'].str.split(' ')
final_movies.head()
final_movies['genre count'] = final_movies.apply(lambda x: len(set(x['genres list']).intersection(fav_genres)), axis=1)
recommendations = final_movies[final_movies['genre count']>0]

ratings['genre list'] = ratings['genres'].str.split(' ')
all_genres_list = []

for i in range(len(movies)):
    all_genres_list.extend(final_movies.iloc[i]['genres list'])

list(set(all_genres_list))

['Fantasy',
 'War',
 'Drama',
 'Western',
 'Crime',
 'Horror',
 'Action',
 'Sci-Fi',
 'Mystery',
 'Animation',
 'Children',
 'Adventure',
 'IMAX',
 'Musical',
 'Romance',
 'Comedy',
 'Thriller']